# IMPORTANT : Please suggest some changes for the line commented using //...// in the markdown

# DeepWalk

Hi, today I will be introducing you to a technique for converting nodes of a given graph to vector, something similar to what is done in the NLP, where words are converted to vector. These vector can be used to extract meaningful insight from the data. 

So lets begin.

Lets assume that you have been given a graph G whose nodes you want to convert to vectors, the only information about a node is the indexes of the nodes to which it is connected. Since there is no initial feature matrix corresponding to the data, we will construct a feature matrix which will have all the randomly selected nodes. There can be multiple methods to select these but here we will be assuming that they are taken from the normal distribution ( though it won't make much difference if they are taken from some other distribution). 

//Since everything is selected out randomly hence this feature matrix will be useles in its presents form. But there is a technique to convert it into something very useful. Thats called the 'DeepWalk'.//

## Intuition

First lets get some intution regarding whats gonna happen. 

Lets deal with an unrelated problem first.

Given some english sentences ( could be any other language, doesn't matter) you have to find a vector corresponding to each word appearing at least once in the sentence such that the words whose meaning is close to each other have their vector similar to each other also, the opposite must hold for the antonym.

Suppose the sentences are
1. Hi I am Sally.
2. Hello this is Sally here.

From the above sentences you can see that the 1 and 3 are pretty related with each other, so even if someone does not know english they can get a rough idea that the words Hi and Hello have roughly the same meaning. We will be using a technique similar to what a human being uses while trying to find out which words are related. Yup, We will be guessing the meaning based on the words which are common between the sentences. Now the question comes, how are we gonna do that.

First lets assign a random vector to each word. Now since these vectors are assigned at random implies the representation in then present form is useless, so how are we gonna make this representation better. Here the good old probability is gonna help us. We will try to maximize the probability of the appearence of a word given the words that appear with it. To maximise the probability, we first need to find out the probability. Lets assume the probability is given by 'P(x | y)' where y is the set of words that appear in the same sentence in which x occurs. Remember we are only taking one sentence at a time, so here first we will try to maximise the probability of 'Hi' given {'I', 'am', 'Sally'} , then we will maximise the probability of 'I' given {'Hi', 'am', 'Sally'}. We will do it for each word in the first sentence , then for the second sentence. Repeating this procedure for all the sentence, and then we will repeat the procedure over all the sentences once again until the feature vector of the words have converged. One question that may arise now is,' How does these feature vectors related with probability?'. The answer is that in the probability function we will not be using the words directly we will actually be using the vectors assinged to them.But aren't those vectors random. Yup they are , but I promise you by the end of the blog they would have converged to values they really assings some meaning to those seamingly random numbers.

First how this probability function works.

What does it mean to find the probability of a vector given other vectors. This is actually a pretty question with a pretty simple answer, Take it as a fill up the blank problem that you dealt in the primary school, Fill up the blank in the given sentence
'Roses ____ red.'. What is the  most likely guess? Most people will fill a 'are' in the blank space (Unless they are pretending to be oversmart in an attempt to prove how cool they are). 

The probability function is also trying to the same it is finding out the word which is most likely to occur given the word that are surrounding it. But it still doesn't tells us how it is gonna do that.

How our brain solve this problem? Hint: It has to do with neurons.

// In case you guessed 'Neural Network' you are correct. In this blog we will also be using neural networs (or a shitty copy of the human brain) ( I know this line was as shit as the neural nets are but I am feeling sleepy and cant think of a better line) ( Please suggest something better).//

//Its not necesary to use the neural networks in the probability funciton but it looks cool so we will be using neural net here.//

The input layer will have |V| neurons, where |V| is the number of words that are interesting to us. We will be using only one hidden layer for simplicity. It can have as many neurons as you want, but I suggest to keep is to be number that is less than the number of words in the vocabulary. The output layer will also have the |V| neurons.

Now lets move on to in interpretation of what the input and output layer symbolises (don't care about the hidden layer).
Lets suppose the words in the vocabulary are V1, V2, ... Vi, .... Vn. Assume that out of these V4,V7, V9 appears along with the word whose probability we are tying to maximise. so the input layers will have the 4th, 7th, and the 9th neuron with value 1 and all other will have the value 0. The hidden layer will then ve some function of these values. The hidden layer have no non linear acitvation. The |V| neuron in the output layer will have a score, the higher it is ,the higher the chances of that word appearning along with the surrounding words. Lets apply a sigmoid to convert these into probabilities. Eureka! we got the probabilities. 


So a simple neural network will help us fill in the blank problem.


But what does it have to do with that so called 'DeepWalk'.
Deepwalk works on graph so this separation based on sentences on graph. It uses a trick called random walk. It will start at the node whose probability we want to find out. Wait Wait Wait, we dont have the surrounding words , how it will find out the probability without them. The answer is that we will find then out in this step of Random Walk. Start at the node, find out all the nodes which have and edge connecting with this start node and randomly select on out of them , then consider this new node as the new start node and repeat the procedue after n iteration you will have traversed n nodes ( some of them might repeat, but it does not matter as in case of sentence words could have repeates). We will take n nodes as the surrounding nodes for the original node ans wil try to maximize probability with respect to those. So that is for you Ladies and Gentlemen , the 'DeepWalk' model.

## IMPLEMENTING The DeepWalk

Here we will use using a the following graph

![title](graph.png)

As you can see there are two connected components, so we can expect than when we create the vectors for each node, the vectors of [1 , 2, 3, 7] should be close and similarly that of [4, 5, 6] should be close. Also if  any two vectors are from different group then their vectors should also be far away.

Here we will we representing the graph using the adjacency list representation. Make sure that you are able to understand that the given graph and this adjacency list are equivalent.

In [1]:
adj_list=[[1,2,3], [0,2,3], [0, 1, 3], [0, 1, 2], [5, 6], [4,6], [4, 5], [1, 3]]

## Imports

In [2]:
import torch
import torch.nn as nn
import random

## Hyperparameter

In [20]:
w=3 #window size
d=2 #embedding size
y=200 #walks per vertex
t=6 #walk length
size_vertex=8 # no of vertices 
lr=0.025 # learning rate

In [4]:
v=[0,1,2,3,4,5,6,7] # labels of available vertices


## Random Walk


In [5]:
def RandomWalk(v,t):
    walk=[v]
    for i in range(t-1):
        v=adj_list[v][random.randint(0,len(adj_list[v])-1)]
        walk.append(v)
    return walk

Here we define the skipgram model.

## Skipgram


The skipgram model is closely related to the CBOW model that we just covered.In the CBOW model we have to maximise the probability of the word given its surrounding word using a neural network. And when the probability is maximised the weights of the input to hidden layer are the word vectors of the given words. In the skipgram word we will be using a using single word to maximise the probability of the surrounding words. This can be done by using a neural network that looks like the mirror image of the network that we used for the CBOW. And in the end the weights of the input to hidden layer will be the corresponding word vectors.


Now lets analyze the complexity.
There are |V| words in the vocabulary so for each iteration we will be modifying a total of |V| vectors. This is bad as usually the vocabulary size is in million and since we usually need millions of iteration before convergence, this can take a long time to run.

We will soon be discussing some methods to reduce this complexity. But lets first look at the code for a simple skipgram model. The class defines the model , whereas the function 'SkipGram' takes care of all the training and other necessary stuff.

In [6]:


class Model(torch.nn.Module):
    def __init__(self):
  
        super(Model, self).__init__()
        self.phi=nn.Parameter(torch.rand((size_vertex, d), requires_grad=True))    
        self.phi2=nn.Parameter(torch.rand((d, size_vertex), requires_grad=True))
    def forward(self, one_hot):
        hidden=torch.matmul(one_hot, self.phi)
        out=torch.matmul(hidden,self.phi2)
            
        return out

In [7]:
model=Model()

In [8]:
def SkipGram(wvi,  w):
   
    for j in range(len(wvi)):
        for k in range(max(0,j-w) , min(j+w, len(wvi))):
            #generate one hot vector
            one_hot=torch.zeros( size_vertex)
            one_hot[wvi[j]]=1
            out=model(one_hot)
            
            loss=  - out[wvi[k]] + torch.log(torch.sum(torch.exp(out)))
            loss.backward()
            for param in model.parameters():
                param.data.sub_(lr*param.grad)
                param.grad.data.zero_()


In [9]:
for i in range(y):
    random.shuffle(v)
    for vi in v:
        wvi=RandomWalk(vi,t)
        SkipGram(wvi, w)

i'th row of the model.phi corresponds to vector of the i'th node. As you can see the vectors of [0, 1, 2,3 , 7] are very close, whereas their vector are much different from the vectors corresponding to [4, 5, 6].

In [10]:
print(model.phi)

Parameter containing:
tensor([[ 0.1972, -1.6208],
        [ 0.6739, -1.3102],
        [ 0.7101, -1.4672],
        [ 0.0476, -1.5772],
        [ 0.5641,  1.8627],
        [ 0.9452,  1.7406],
        [ 1.0284,  1.6569],
        [ 0.1367, -0.3120]], requires_grad=True)


Above we defined and used the skipgram model. Now we will be discussing a variant known as the Hierarchical softmax.

## Hierarchical Softmax

As we have seen in the skip-gram model that the probability of any outcome depends on the total outcomes of model. If you haven't noticed, let me explain you how! Since when we calculate the probability of an outcomes using softmax, this probability depends on the number of model parameters via the normalisation constant(denominator term) in the softmax. And the number of such parameters is linear in the total number of outcomes. It means if we are dealing with a very large graphical structure, it can computationally expensive and can take a lot of time. Can we somehow overcome this challenge? Obviously yes!( because I'm asking at this stage). We can overcome this problem using "hierarchical softmax".Hierarchical softmax is an alternative to the softmax in which the probability of any one outcome depends on a number of model parameters that is only logarithmic in the total number of outcomes. 

Let's dive deep into Hierarchical softmax.

Hierarchical softmax uses a binary tree to represent all words in the vocabulary. Each leaf of the tree is a node of our graph, and there is a unique path from root to leaf.Each intermediate node of tree explicitly represents the relative probabilities of its child nodes. So these nodes are associated to different certain vectors which our model is going to learn.

The idea behind decomposing the output layer into binary tree is to reduce the time complexity to obtain 
probability distribution from O(V) to O(log(V))

Let us understand the process with an example.

![binary tree](tree.png)

In this example, leaf nodes represent the original nodes of our graph.The highlighted nodes and edges make a path from root to an example leaf node w2.

Here length of the path L(w2) = 4.

n(w, j) means the jth node on the path from root to a leaf node w.

Now view this tree as a decision process, or a random walk, that begins at the root of the tree and descents towards the leaf nodes at each step. It turns out that the probability of each outcome in the original distribution uniquely determines the transition probabilities of this random walk.If you want to go from root node to w2(say), first you have to take a left turn, again left turn and then right turn. 

Let's denote the probability of going left at an intermediate node n as p(n,left) and probability of going right as p(n,right). So we can define the probabilty of going to w2 as follows.
#### P(w2|wi) = p(n(w2,1),left) . p(n(w2,2),left) . p(n(w2,3),right) .

Above process implies that the cost for computing the loss function and its gradient will be proportional to the number of nodes (V) in the intermediate path between root node and the output node, which on average is no greater than log (V). That's nice! Isn't it? In the case where we deal with a large number of outcomes, there will be a huge difference in the computational cost of 'vanilla' softmax and hierarchical softmax.

Code is similar to the above part , except that we will only need to change to Model class by HierarchicalModel class, which is given below.

In [11]:
#func_L returns the length of path from the root node to the given vertex

def func_L(w):
    cnt=1
    while(w!=1):
        cnt+=1
        w//=2
    return cnt

In [12]:
#func_n return the nth node in the path from the root node to the given vertex
def func_n(w, j):
    li=[w]
    while(w!=1):
        w=w//2;
        li.append(w)
    li.reverse()
    return li[j]

In [13]:
def sigmoid(x):
    return 1/(1+torch.exp(-x))

In [14]:
class HierarchicalModel(torch.nn.Module):
    def __init__(self):
  
        super(HierarchicalModel, self).__init__()
        self.phi=nn.Parameter(torch.rand((size_vertex, d), requires_grad=True))   
        self.prob_tensor=nn.Parameter(torch.rand((2*size_vertex, d), requires_grad=True))
    def forward(self, wi, wo):
        one_hot=torch.zeros( size_vertex)
        one_hot[wi]=1
        w=size_vertex+wo
        h=torch.matmul(one_hot,self.phi)
        p=torch.tensor([1.0])
        for j in range(1,func_L(w)-1):
            mult=-1
            if(func_n(w, j+1)==2*func_n(w, j)): #Left child
                mult=1
            p=p*sigmoid(mult*torch.matmul(self.prob_tensor[func_n(w,j)], h))
        
        return p

The input to hidden weight vector no longer represents the vector corresponding to each vector , so directly trying to read it will not provide any valuable insight, a better option is to predict the probability of different vectors against each other to figure out the likelihood of coexistance of the nodes.

In [15]:
hierarchicalModel=HierarchicalModel()

In [16]:
def HierarchicalSkipGram(wvi,  w):
   
    for j in range(len(wvi)):
        for k in range(max(0,j-w) , min(j+w, len(wvi))):
            #generate one hot vector
       
            prob=hierarchicalModel(wvi[j], wvi[k])
            loss=  - torch.log(prob)
            loss.backward()
            for param in hierarchicalModel.parameters():
                param.data.sub_(lr*param.grad)
                param.grad.data.zero_()


In [21]:
for i in range(y):
    random.shuffle(v)
    for vi in v:
        wvi=RandomWalk(vi,t)
        HierarchicalSkipGram(wvi, w)

In [22]:
for i in range(8):
    for j in range(8):
        print((hierarchicalModel(i,j).item()*100)//1, end=' ')
    print(end='\n')

31.0 30.0 14.0 22.0 19.0 9.0 3.0 67.0 
25.0 34.0 15.0 23.0 16.0 8.0 0.0 75.0 
19.0 28.0 25.0 26.0 20.0 20.0 1.0 56.0 
23.0 33.0 18.0 24.0 17.0 10.0 0.0 71.0 
38.0 16.0 23.0 22.0 33.0 33.0 33.0 0.0 
27.0 18.0 29.0 23.0 28.0 38.0 32.0 0.0 
32.0 19.0 23.0 23.0 30.0 30.0 38.0 0.0 
22.0 37.0 15.0 24.0 13.0 6.0 0.0 80.0 
